# Lock exchange flow example
This is a test case with particles released in a simple looping flow field 
The background flow velocity is given by a lock exchange flow.

In [1]:
# Load required libraries
using Pkg
Pkg.instantiate()
Pkg.activate("..")
using Plots
using Particles
using Random
using NetCDF
nothing

  Activating project at `j:\Master_Thesis\particles.jl`


In [3]:
randpool = MersenneTwister(0) #Generate same random numbers every time

# load useful default settings 
d=default_userdata()
n=10 #number of particles

# general setting
d["coordinates"] = "projected"
d["nparticles"]= 10 #number of particles
d["time_direction"] = :forwards # :forwards or :backwards
d["variables"] = ["x","y","z","age"] # if not stated otherwise explicitly

# load data from nc file 
dflow_map = load_nc_info(@__DIR__, r"locxx_map.nc")
# construct a grid for interpolation
# 2nd argument is number of grid points per dimension
# 3rd argument to supress spherical coordinates
interp = load_dflow_grid(dflow_map, 50, false); 

# set simulation time
# read the reftime from the file
d["reftime"] = get_reftime(dflow_map)
t_ref = get_reftime(dflow_map)
rel_times = get_times(dflow_map, t_ref)
d["dt"] = rel_times[2] - rel_times[1]
d["tstart"] = rel_times[begin]
d["tend"] = rel_times[end]

# particle settings
xnodes = dflow_map[1].vars["mesh2d_node_x"]
ynodes = dflow_map[1].vars["mesh2d_node_y"]
xmin = minimum(xnodes); xmax = maximum(xnodes);
ymin = minimum(ynodes); ymax = maximum(ynodes);
p0 = zeros(length(d["variables"]), d["nparticles"])
p0[1, :] = 0.5*(xmin+xmax)*(1 .+ rand(randpool, d["nparticles"])) 
p0[2, :] = 0.5*(ymin+ymax)*(1 .+ rand(randpool, d["nparticles"]))
d["particles"] = copy(p0)

# record grid range in each dimension
d["bbox"] = (xmin, ymin, xmax, ymax)

# plot map times 
d["plot_maps_times"] = rel_times                        # Time at which plot should be made
d["plot_maps"] = false # background flow is time evolving
                       # not suitable for using background function anymore
d["plot_maps_folder"] = "maps"
d["plot_maps_size"]=(2000,600)

# plot particle times
d["keep_particle_times"] = rel_times                        # Time at which plot should be made
d["keep_particles"] = true


compute index:

┌ Info: j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc
└ @ Particles J:\Master_Thesis\particles.jl\src\dflow.jl:27



- j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc


true

In [4]:
function load_nc_dims(ncFile_name)
    nc = NetCDF.open(ncFile_name, readdimvar=true)
    dims = Dict()
    for k in keys(nc.dim)
        display(k)
        dims[k] = Int(nc.dim[k].dimlen)
    end
    return dims
end
# return a dict that stores the name and length of different dimensions
nc_dims = load_nc_dims("locxx_map.nc") 


"mesh2d_nNodes"

"Two"

"mesh2d_nInterfaces"

"time"

"mesh2d_nMax_face_nodes"

"mesh2d_nFaces"

"mesh2d_nLayers"

"mesh2d_nEdges"

Dict{Any, Any} with 8 entries:
  "mesh2d_nNodes"          => 802
  "Two"                    => 2
  "mesh2d_nInterfaces"     => 41
  "time"                   => 21
  "mesh2d_nMax_face_nodes" => 4
  "mesh2d_nFaces"          => 400
  "mesh2d_nLayers"         => 40
  "mesh2d_nEdges"          => 1201

In [5]:
ncFile_name = "locxx_map.nc"
nc = NetCDF.open(ncFile_name, readdimvar=true)
display(nc.dim) # a dict with a key of String type and a value of NcDim type
# return a dict that stores the name and length of different dimensions using user-defined functions
nc_dims = load_nc_dims("locxx_map.nc") 
display(nc.vars) # a dict with a key of String type and an array of values

Dict{String, NcDim} with 8 entries:
  "mesh2d_nNodes"          => NcDim(196608, 2, -1, "mesh2d_nNodes", 0x000000000…
  "Two"                    => NcDim(196608, 0, -1, "Two", 0x0000000000000002, A…
  "mesh2d_nInterfaces"     => NcDim(196608, 6, -1, "mesh2d_nInterfaces", 0x0000…
  "time"                   => NcDim(196608, 7, 19, "time", 0x0000000000000015, …
  "mesh2d_nMax_face_nodes" => NcDim(196608, 4, -1, "mesh2d_nMax_face_nodes", 0x…
  "mesh2d_nFaces"          => NcDim(196608, 3, -1, "mesh2d_nFaces", 0x000000000…
  "mesh2d_nLayers"         => NcDim(196608, 5, -1, "mesh2d_nLayers", 0x00000000…
  "mesh2d_nEdges"          => NcDim(196608, 1, -1, "mesh2d_nEdges", 0x000000000…

"mesh2d_nNodes"

"Two"

"mesh2d_nInterfaces"

"time"

"mesh2d_nMax_face_nodes"

"mesh2d_nFaces"

"mesh2d_nLayers"

"mesh2d_nEdges"

Dict{String, NcVar} with 46 entries:
  "mesh2d_ucx"                  => Disk Array with size 40 x 400 x 21…
  "mesh2d_taus"                 => Disk Array with size 400 x 21…
  "mesh2d_edge_nodes"           => Disk Array with size 2 x 1201…
  "mesh2d_u0"                   => Disk Array with size 40 x 1201 x 21…
  "mesh2d_Numlimdt"             => Disk Array with size 400 x 21…
  "mesh2d_tureps1"              => Disk Array with size 41 x 1201 x 21…
  "mesh2d_q1"                   => Disk Array with size 40 x 1201 x 21…
  "mesh2d_interface_z"          => Disk Array with size 41…
  "mesh2d_edge_y"               => Disk Array with size 1201…
  "mesh2d_face_y"               => Disk Array with size 400…
  "mesh2d_edge_faces"           => Disk Array with size 2 x 1201…
  "projected_coordinate_system" => Disk Array with size …
  "mesh2d_u1"                   => Disk Array with size 40 x 1201 x 21…
  "mesh2d_s0"                   => Disk Array with size 400 x 21…
  "time"                        =

In [6]:
d["time_direction"] = :forwards # :forwards or :backwards


:forwards

In [11]:
display(maximum(dflow_map[1].vars["mesh2d_node_z"]))
minimum(dflow_map[1].vars["mesh2d_node_z"])
dflow_map[1].vars["mesh2d_node_z"]


-999.0

Disk Array with size 802


In [12]:
display(nc.vars["mesh2d_face_x"][:])

400-element Vector{Float64}:
 299.62500000000006
 298.87500000000006
 298.12500000000006
 297.37500000000006
 296.62500000000006
 295.87500000000006
 295.12500000000006
 294.37500000000006
 293.62500000000006
 292.87500000000006
   ⋮
   6.375000000000001
   5.625000000000001
   4.875000000000001
   4.125000000000001
   3.3750000000000004
   2.6250000000000004
   1.8750000000000004
   1.1250000000000002
   0.37500000000000006

In [13]:
temp = dflow_map[1].vars["mesh2d_node_x"]



299.25000000000006

0.0

In [9]:
# # check default reftime
# println(d["reftime"])
# # read reftime and times using lib functions
# t_ref = get_reftime(dflow_map)
# println(t_ref)
# println(d["reftime"] == t_ref)
# times = get_times(dflow_map, t_ref)
# # read times, equivalent to using lib functions
# time_relative = dflow_map[1].vars["time"]
# units = time_relative.atts["units"]
# println(time_relative == times)
# println(time_relative == dflow_map[1]["time"][:])

In [15]:
# prepare the interpolator for velocity field
# no need for the 3rd dimension 
u = initialize_interpolation(dflow_map, interp, "mesh2d_ucx", d["reftime"], 0.0, d["time_direction"]);
v = initialize_interpolation(dflow_map, interp, "mesh2d_ucy", d["reftime"], 0.0, d["time_direction"]);
display(typeof(u))


Particles.var"#f#25"{Interpolator, Float64, Symbol, Particles.var"#weights#24", Particles.var"#update_cache_backwards#22"{Vector{Any}}, Particles.var"#update_cache_forwards#19"{Vector{Int64}, Vector{Any}, Bool, Int64}, Vector{Any}}

initialize caching for j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucx...
initialize caching for j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucy...


In [ ]:
"""
# plot with user defined function
# not good becuase it's inefficient
gr(legend=false)

# as: arrow head size 0-1 (fraction of arrow length)
# la: arrow alpha transparency 0-1
function arrow0!(x, y, u, v; as=0.07, lc=:black, la=1)
    nuv = sqrt(u^2 + v^2)
    v1, v2 = [u;v] / nuv,  [-v;u] / nuv
    v4 = (3*v1 + v2)/3.1623  # sqrt(10) to get unit vector
    v5 = v4 - 2*(v4'*v2)*v2
    v4, v5 = as*nuv*v4, as*nuv*v5
    plot!([x,x+u], [y,y+v], lc=lc,la=la)
    plot!([x+u,x+u-v5[1]], [y+v,y+v-v5[2]], lc=lc, la=la)
    plot!([x+u,x+u-v4[1]], [y+v,y+v-v4[2]], lc=lc, la=la)
end

# explicitly prepare the grid for ploting
scale = 10
x_resolution=21 #[0,300]
y_resolution=11   #[0,1]
x = collect(range(minimum(dflow_map[1]["mesh2d_node_x"][:]), maximum(dflow_map[1]["mesh2d_node_x"][:]), length=x_resolution))
y = collect(range(minimum(dflow_map[1]["mesh2d_node_y"][:]), maximum(dflow_map[1]["mesh2d_node_y"][:]), length=y_resolution))
xs=kron(x,ones(length(y)))
ys=kron(ones(length(x)),y)

Plots.default(:size, d["plot_maps_size"])
u_plot = u.(xs, ys, 0, 100)
v_plot = v.(xs, ys, 0, 100)
#p = quiver(xs, ys, quiver=(scale*u_plot, scale*v_plot))
#savefig(p, "quiver.png") 
for (x,y,u,v) in zip(xs, ys, u_plot,v_plot)
    display(arrow0!(x,y,u,v;as=0.1, lc:=black, la=1))
end
"""

ErrorException: syntax: invalid keyword argument syntax "lc := black" around j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_org.ipynb:33

In [ ]:
# explicitly prepare the grid for ploting
scale = 10
x_resolution=31 #[0,300]
y_resolution=31   #[0,1]
x = collect(range(minimum(dflow_map[1]["mesh2d_node_x"][:]), maximum(dflow_map[1]["mesh2d_node_x"][:]), length=x_resolution))
y = collect(range(minimum(dflow_map[1]["mesh2d_node_y"][:]), maximum(dflow_map[1]["mesh2d_node_y"][:]), length=y_resolution))
xs=kron(x,ones(length(y)))
ys=kron(ones(length(x)),y)

Plots.default(:size, d["plot_maps_size"])
u_plot = u.(xs, ys, 0, 100)
v_plot = v.(xs, ys, 0, 100)
#p = quiver(xs, ys, quiver=(scale*u_plot, scale*v_plot))
#savefig(p, "quiver.png") 

In [12]:
###### Velocity function for the particles ######
function f1!(ds, s, t, i, d)
   x, y, z, age = s
   z = 0.0
   up = 0
   vp = 0
   dt = d["dt"]
   uw = u(x, y, z, t)
   vw = v(x, y, z, t)

   # The model with :
   # 1: Only flow velocities
   up = uw
   vp = vw
   # original computation for computing the particle velocity
   #(K, Kdx, Kdy) = estimate_viscosity_smag(interp, x, y, t, u, v)
   #if !(uw == vw == ua == va == 0.0)
   #   # https://doi.org/10.1016/j.ocemod.2017.11.008 eq. 27
   #   up += Kdy + randn() * sqrt(2 * K * dt) / dt
   #   vp += Kdx + randn() * sqrt(2 * K * dt) / dt
   #end
   
   epsx = 0#1
   epsz = 0#1e-5
   up += randn()*sqrt(2*epsx*dt)/dt
   vp += randn()*sqrt(2*epsz*dt)/dt

   ds.x = up
   ds.y = vp
   ds.z = 0.0
   ds.t = 1.0


   if d["time_direction"] == :backwards
      up *= -1
      vp *= -1
   end
end
d["f"]=f1!

f1! (generic function with 1 method)

In [13]:
# use streamfunction as background for plotting
function plot_vfield(d,t)
    x_resolution=41 #[0,300]
    y_resolution=21   #[0,1]
    x_min, y_min, x_max, y_max = d["bbox"]
    x = collect(range(x_min, x_max, length=x_resolution))
    y = collect(range(y_min, y_max, length=y_resolution))
    xs=kron(x,ones(length(y)))
    ys=kron(ones(length(x)),y)
    Plots.default(:size, d["plot_maps_size"])
    u_plot = u.(xs, ys, 0, 100)
    v_plot = v.(xs, ys, 0, 100)
    p = quiver(xs, ys, quiver=(scale*u_plot, scale*v_plot))
    return p
 end
 d["plot_vfield"] = plot_vfield 

plot_vfield (generic function with 1 method)

In [14]:
using Printf
# screenshot of particles
function plot_screenshot(d; dirname="", fnametag="")
    t_plot=d["keep_particle_times"];
    p=d["all_particles"]
    for i in 1:length(t_plot)
        fig = d["plot_vfield"](d, t_plot[i])
        fname = dirname* @sprintf("\\screenshot_time %.2f",t_plot[i]) *".png"
        d["plot_maps_func"](fig, d, p[i])
        savefig(fig, fname)
    end
  end
  
#   function plot_trajectories(d; dirname="", fnametag="")
#       # plot particles' trajectories
#       t_plot = d["keep_particle_times"];
#       p = d["all_particles"]
#       n = d["nparticles"]
#       p_x = zeros(length(t_plot), n)
#       p_y = zeros(length(t_plot), n)
#       for i in 1:length(t_plot)
#           p_x[i, :] = p[i][1,:]
#           p_y[i, :] = p[i][2,:]
#       end
  
#       # prepare the data for plotting
#       # note that the data points at the first and the end are used once
#       indices = reduce(vcat, [[j for i in 1:2] for j in 1:length(t_plot)]) #reduce a vector of vectors to a vector
#       indices = indices[begin+1:end-1]
#       fig = d["plot_maps_background"](d, t_plot[i])
#       fname = dirname*"\\trajectories_$(fnametag).png"
  
#       for i in 1:n
#           plot!(fig, p_x[indices, i], p_y[indices, i], 
#                  legend=egend =:outerright, 
#                  size = (1400, 800),
#                  linewidth = 2.5,
#                  markershape = :circle,
#                  markersize = 1.5,
#                  markerstrokecolor=:auto,
#                  label = "particle$(i)")
#       end
  
#       savefig(fig, fname)
#   end

plot_screenshot (generic function with 1 method)

In [15]:
dirname = "plots"
if ~isdir(dirname)
    mkdir(dirname)
end
dirname = joinpath(@__DIR__, dirname)

"j:\\Master_Thesis\\particles.jl\\case_lock_exchange\\plots"

In [16]:
open("stdout.txt","w") do io
    redirect_stdout(io) do
        @time run_simulation(d)
        plot_screenshot(d; dirname=dirname)
    end
end

In [17]:
display(d["all_particles"][1])
display(d["all_particles"][2])

4×10 Matrix{Float64}:
 273.547     286.553     174.685     …  160.24      204.274   295.982
   0.792906    0.769645    0.630018       0.934139    0.9839    0.883845
   0.0         0.0         0.0            0.0         0.0       0.0
   0.0         0.0         0.0            0.0         0.0       0.0

4×10 Matrix{Float64}:
 273.547     286.553     174.685     …  160.24      204.274   295.982
   0.792906    0.769645    0.630018       0.934139    0.9839    0.883845
   0.0         0.0         0.0            0.0         0.0       0.0
  30.0        30.0        30.0           30.0        30.0      30.0